In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'
import sys
sys.path.append("../.venv/lib/python3.9/site-packages/")
sys.path.append("..")

In [182]:
from typing import Any, List

import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import ttest_rel, ttest_ind, norm
from sklearn.base import BaseEstimator
from sklearn.linear_model import BayesianRidge

from coordination.model.beta_coordination_blending_latent_vocalics import BetaCoordinationBlendingLatentVocalics
from coordination.model.utils.coordination_blending_latent_vocalics import LatentVocalicsDataset
from scripts.formatting import set_size

In [3]:
PLOTS_DIR = "/Users/paulosoares/manuscript/experimental_agenda/figures"

def save_plot(fig: Any, name: str):
    fig.savefig(f"{PLOTS_DIR}/{name}.pdf", format='pdf', bbox_inches='tight')

In [202]:
def load_datasets(advisor: str):
    with open(f"/Users/paulosoares/data/study-3_2022/datasets/2_features/{advisor}/mission1_dataset.pkl", "rb") as f:
        mission1_dataset = pickle.load(f)
    
    with open(f"/Users/paulosoares/data/study-3_2022/datasets/2_features/{advisor}/mission2_dataset.pkl", "rb") as f:
        mission2_dataset = pickle.load(f)
    
    with open(f"/Users/paulosoares/data/study-3_2022/datasets/2_features/{advisor}/all_missions_dataset.pkl", "rb") as f:
        all_missions_dataset = pickle.load(f)
    
    return mission1_dataset, mission2_dataset, all_missions_dataset

def load_model_inferences(advisor: str, ref_date: str, training_type: str):
    with open(f"/Users/paulosoares/data/study-3_2022/inferences/beta_gaussian/{advisor}/{training_type}/mission1/{ref_date}/inference_summaries.pkl", "rb") as f:
        mission1_summaries = pickle.load(f)
    
    with open(f"/Users/paulosoares/data/study-3_2022/inferences/beta_gaussian/{advisor}/{training_type}/mission2/{ref_date}/inference_summaries.pkl", "rb") as f:
        mission2_summaries = pickle.load(f)
    
    with open(f"/Users/paulosoares/data/study-3_2022/inferences/beta_gaussian/{advisor}/{training_type}/all_missions/{ref_date}/inference_summaries.pkl", "rb") as f:
        all_missions_summaries = pickle.load(f)
    
    return mission1_summaries, mission2_summaries, all_missions_summaries

def cohens_d(x1: np.ndarray, x2: np.ndarray):
    n1 = len(x1)
    n2 = len(x2)
    sp = np.sqrt(((n1 - 1) * np.var(x1) + (n2 - 1) * np.var(x2))/ (n1 + n2 - 2))
    
    return (np.mean(x1) - np.mean(x2)) / sp    

def compare_conditions(x1: np.ndarray, x2: np.ndarray, alternative: str = "greater", paired: bool = False):
    d = cohens_d(x1, x2)
    
    if paired:
        _, p_val = ttest_rel(x1, x2, alternative=alternative)
    else:
        _, p_val = ttest_ind(x1, x2, alternative=alternative)
    
    results = {
        "cohens_d": d, 
        "p_val": p_val
    }    
    
    return results

In [203]:
# Loading all datasets
no_advisor_datasets = load_datasets("no_advisor")
human_advisor_datasets = load_datasets("human_advisor")
tomcat_advisor_datasets = load_datasets("tomcat_advisor")
all_advisors_datasets = load_datasets("all_advisors")

datasets = {
    "no_advisor": load_datasets("no_advisor"),
    "human_advisor": load_datasets("human_advisor"),
    "tomcat_advisor": load_datasets("tomcat_advisor"),
    "all_advisors": load_datasets("all_advisors")
}

In [65]:
# All models and conditions we want to evaluate
models = [
    {"ref_date":"2022.12.02--15", "training_type":"single_execution"},
    {"ref_date":"2022.12.04--12", "training_type":"single_execution_no_self_dep"},
    {"ref_date":"2022.12.04--22", "training_type":"single_execution_intensity_only"},
    {"ref_date":"2022.12.05--09", "training_type":"single_execution_pitch_only"}
]

In [15]:
from functools import partial
AGGR_FN = np.mean
# AGGR_FN = np.max
# AGGR_FN = np.var
# AGGR_FN = partial(np.percentile, q=75)

# Hypothesis I: Predictive Power

Coordination is predictive of outcome measure ($M_i$). For each outcome measure, we train Bayesian linear model that tells us p($M_i$ | $C$) in a model with coordination and in a model without it. We then compute MSE and log-likelihood in a holdout set (using LOO) and report the average. We also compute how many times the model with coordination was better than the model without. We accept the hypothesis that the model with coordination is better than the one without if it is better 95% of the time.

In [100]:
class NullModel(BaseEstimator):
    
    def __init__(self):
        self.mean = None
        self.std = None
    
    def fit(self, X: np.ndarray, y: Any = None):
        self.mean = np.mean(y, axis=0)
        self.std = np.mean(y, axis=0)
        return self
    
    def predict(self, X: np.ndarray):
        return np.ones(X.shape[0]) * self.mean
    
    def compute_log_likelihood(self, X: np.ndarray, y: np.ndarray, sample_weight=None):
        return norm(loc=self.mean, scale=self.std).logpdf(y)

class CoordinationModel(BaseEstimator):
    
    def __init__(self):
        self.reg = BayesianRidge(tol=1e-6, fit_intercept=True)
    
    def fit(self, X: np.ndarray, y: Any = None):
        self.reg.fit(X, y)
        return self
    
    def predict(self, X: np.ndarray):
        return self.reg.predict(X)
    
    def compute_log_likelihood(self, X: np.ndarray, y: np.ndarray, sample_weight=None):
        mean, std = self.reg.predict(X, return_std=True)
        return norm(loc=mean, scale=std).logpdf(y)

def execute_loo(X: np.ndarray, y: np.ndarray):
    null_model = NullModel()
    coord_model = CoordinationModel()
    null_mses = []
    null_nlls = []
    coord_mses = []
    coord_nlls = []
    
    loo = LeaveOneOut()
    for train_index, test_index in loo.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        null_model.fit(X_train, y_train)
        null_mses.append(mean_squared_error(null_model.predict(X_test), y_test))
        null_nlls.append(-null_model.compute_log_likelihood(X_test, y_test)[0])   

        coord_model.fit(X_train, y_train)
        coord_mses.append(mean_squared_error(coord_model.predict(X_test), y_test))
        coord_nlls.append(-coord_model.compute_log_likelihood(X_test, y_test)[0])
    
    null_mses = np.array(null_mses)
    coord_mses = np.array(coord_mses)
    null_nlls = np.array(null_nlls)
    coord_nlls = np.array(coord_nlls)
    
    _, p_val_nll = ttest_rel(coord_nlls, null_nlls, alternative="less")
    _, p_val_mse = ttest_rel(coord_mses, null_mses, alternative="less")
    
    results = {
        "avg_mse_null": np.mean(null_mses),
        "std_mse_null": np.std(null_mses),
        "avg_nll_null": np.mean(null_nlls),
        "std_nll_null": np.std(null_nlls),
        "avg_mse_coord": np.mean(coord_mses),
        "std_mse_coord": np.std(coord_mses),
        "avg_nll_coord": np.mean(coord_nlls),
        "std_nll_coord": np.std(coord_nlls),    
        "p_coord_smaller_mse": (100.0 * np.sum(coord_mses < null_mses)) / len(coord_mses),
        "p_coord_smaller_nll": (100.0 * np.sum(coord_nlls < null_nlls)) / len(coord_nlls),
        "p_val_nll": p_val_nll,
        "p_val_mse": p_val_mse,
        "cohens_d_nll": cohens_d(coord_nlls, null_nlls),
        "cohens_d_mse": cohens_d(coord_mses, null_mses),
    }
    
    return results

In [145]:
outcome_measures_labels = ["Score", "Process Scale", "Satisfaction"]

results_list = []
for condition in ["no_advisor", "human_advisor", "tomcat_advisor", "all_advisors"]:
    for model in models:
        for i, mission in enumerate(["Mission 1", "Mission 2", "All Missions"]):
            dataset = datasets[condition][i]
            inferences = load_model_inferences(condition, model["ref_date"], model["training_type"])            
            X = np.array([AGGR_FN(summary.coordination_mean) for summary in inferences[i]])[:, np.newaxis]
            outcome_measures = [dataset.team_scores, 
                                np.mean(dataset.team_process_surveys, axis=(1,2)),
                                np.mean(dataset.team_satisfaction_surveys, axis=(1,2))]
            
            for j, outcome_measure in enumerate(outcome_measures):
                results_entry = model.copy()
                results_entry.update({
                    "condition": condition,
                    "mission": mission,
                    "outcome_measure": outcome_measures_labels[j]
                })
                results_entry.update(execute_loo(X, outcome_measure))
                results_list.append(results_entry)

df_h1 = pd.DataFrame.from_dict(results_list)
df_h1.head()

,ref_date,training_type,condition,mission,outcome_measure,avg_mse_null,std_mse_null,avg_nll_null,std_nll_null,avg_mse_coord,std_mse_coord,avg_nll_coord,std_nll_coord,p_coord_smaller_mse,p_coord_smaller_nll,p_val_nll,p_val_mse,cohens_d_nll,cohens_d_mse
0,2022.12.02--15,single_execution,no_advisor,Mission 1,Score,30865.289256,42360.147507,7.274091,0.078340,36128.573741,43301.500621,6.643223,0.601163,66.666667,83.333333,1.362486e-03,0.924711,-1.471652,0.122878
1,2022.12.02--15,single_execution,no_advisor,Mission 1,Process Scale,0.230725,0.349627,2.312507,0.010497,0.257579,0.366515,0.997525,1.283109,25.000000,91.666667,2.917786e-03,0.902178,-1.449295,0.074975
2,2022.12.02--15,single_execution,no_advisor,Mission 1,Satisfaction,0.269862,0.329290,2.284625,0.011788,0.308654,0.382821,0.988979,0.694353,33.333333,91.666667,3.256959e-05,0.881155,-2.638510,0.108642
3,2022.12.02--15,single_execution,no_advisor,Mission 2,Score,14790.532544,15815.577307,7.477119,0.019822,16143.201445,20273.257408,6.458972,0.443519,64.285714,92.857143,5.810999e-07,0.762226,-3.243242,0.074398
4,2022.12.02--15,single_execution,no_advisor,Mission 2,Process Scale,0.171687,0.186821,2.412523,0.009827,0.182391,0.245938,0.749075,0.396103,50.000000,100.000000,4.761438e-10,0.661223,-5.937215,0.049013


In [171]:
df_h1[(df_h1["p_val_nll"] <= 0.05) & (df_h1["training_type"] == "single_execution") & (df_h1["p_coord_smaller_nll"] >= 95)].reset_index()

,index,ref_date,training_type,condition,mission,outcome_measure,avg_mse_null,std_mse_null,avg_nll_null,std_nll_null,avg_mse_coord,std_mse_coord,avg_nll_coord,std_nll_coord,p_coord_smaller_mse,p_coord_smaller_nll,p_val_nll,p_val_mse,cohens_d_nll,cohens_d_mse
0,4,2022.12.02--15,single_execution,no_advisor,Mission 2,Process Scale,0.171687,0.186821,2.412523,0.009827,0.182391,0.245938,0.749075,0.396103,50.000000,100.000000,4.761438e-10,0.661223,-5.937215,0.049013
1,5,2022.12.02--15,single_execution,no_advisor,Mission 2,Satisfaction,0.186824,0.200787,2.383660,0.009744,0.199229,0.210657,0.681663,0.588756,57.142857,100.000000,4.734613e-08,0.962598,-4.087699,0.060279
2,7,2022.12.02--15,single_execution,no_advisor,All Missions,Process Scale,0.233596,0.232209,2.368501,0.007881,0.233624,0.232224,0.717980,0.583981,42.307692,96.153846,7.733463e-14,0.957288,-3.996663,0.000121
3,8,2022.12.02--15,single_execution,no_advisor,All Missions,Satisfaction,0.254756,0.262432,2.339950,0.009171,0.257966,0.266053,0.759159,0.597222,53.846154,96.153846,3.178292e-13,0.837651,-3.742855,0.012149
4,39,2022.12.02--15,single_execution,human_advisor,Mission 2,Score,14144.378698,12626.848705,7.433515,0.020887,14945.136153,13343.041868,6.280736,0.297304,50.000000,100.000000,9.743903e-10,0.625528,-5.470049,0.061645
5,42,2022.12.02--15,single_execution,human_advisor,All Missions,Score,18796.320000,19847.532702,7.366481,0.028349,19707.875763,20672.920122,6.424590,0.375151,65.384615,96.153846,2.934304e-13,0.759861,-3.540568,0.044983
6,43,2022.12.02--15,single_execution,human_advisor,All Missions,Process Scale,0.182968,0.226961,2.307057,0.008520,0.182989,0.226992,0.609041,0.717761,42.307692,96.153846,3.904213e-12,0.985276,-3.345386,0.000089
7,73,2022.12.02--15,single_execution,tomcat_advisor,Mission 1,Process Scale,0.066150,0.041415,2.281602,0.005631,0.085496,0.059106,0.250187,0.288453,58.333333,100.000000,4.790623e-11,0.842657,-9.957611,0.379104
8,74,2022.12.02--15,single_execution,tomcat_advisor,Mission 1,Satisfaction,0.092121,0.103869,2.248328,0.009068,0.096349,0.110934,0.416368,0.557480,50.000000,100.000000,1.359223e-07,0.837268,-4.646696,0.039341
9,75,2022.12.02--15,single_execution,tomcat_advisor,Mission 2,Score,39701.388889,35192.945095,7.353551,0.056008,19017.153941,25871.263907,6.532567,0.304359,76.923077,100.000000,3.145511e-07,0.038947,-3.751741,-0.669700


# Hypothesis II: Increased coordination predicts higher outcome measures

Perform a p-test over samples from the posterior distribution of slopes and check if that is bigger than 0. Moreover, compute effect size using Cohen's d measurement.

In [144]:
def test_positive_slope(X: np.ndarray, y: np.ndarray, num_samples: int):
    reg = BayesianRidge(tol=1e-6, fit_intercept=True)
    reg.fit(X, y)
    
    mean = reg.coef_[0]
    std = np.sqrt(reg.sigma_.flatten()[0])
    slope_posterior = norm(loc=mean, scale=std)
    
    slope_samples = slope_posterior.rvs(num_samples)
    d = cohens_d(slope_samples, np.zeros(num_samples))
    _, p_val = ttest_ind(slope_samples, np.zeros(num_samples), alternative="greater")
    
    results = {
        "cohens_d": d, 
        "p_val": p_val
    }    
    
    return results

In [147]:
outcome_measures_labels = ["Score", "Process Scale", "Satisfaction"]

results_list = []
for condition in ["no_advisor", "human_advisor", "tomcat_advisor", "all_advisors"]:
    for model in models:
        for i, mission in enumerate(["Mission 1", "Mission 2", "All Missions"]):
            dataset = datasets[condition][i]
            inferences = load_model_inferences(condition, model["ref_date"], model["training_type"])            
            X = np.array([AGGR_FN(summary.coordination_mean) for summary in inferences[i]])[:, np.newaxis]
            outcome_measures = [dataset.team_scores, 
                                np.mean(dataset.team_process_surveys, axis=(1,2)),
                                np.mean(dataset.team_satisfaction_surveys, axis=(1,2))]
            
            for j, outcome_measure in enumerate(outcome_measures):
                results_entry = model.copy()
                results_entry.update({
                    "condition": condition,
                    "mission": mission,
                    "outcome_measure": outcome_measures_labels[j]
                })
                results_entry.update(test_positive_slope(X, outcome_measure, 10000))
                results_list.append(results_entry)

df_h2 = pd.DataFrame.from_dict(results_list)
df_h2.head()

,ref_date,training_type,condition,mission,outcome_measure,cohens_d,p_val
0,2022.12.02--15,single_execution,no_advisor,Mission 1,Score,-1.346177,1.0
1,2022.12.02--15,single_execution,no_advisor,Mission 1,Process Scale,-0.611089,1.0
2,2022.12.02--15,single_execution,no_advisor,Mission 1,Satisfaction,-0.764306,1.0
3,2022.12.02--15,single_execution,no_advisor,Mission 2,Score,1.939524,0.0
4,2022.12.02--15,single_execution,no_advisor,Mission 2,Process Scale,2.096874,0.0


In [173]:
df_h2[(df_h2["p_val"] <= 0.05) & (df_h2["training_type"] == "single_execution")]

,ref_date,training_type,condition,mission,outcome_measure,cohens_d,p_val
3,2022.12.02--15,single_execution,no_advisor,Mission 2,Score,1.939524,0.000000e+00
4,2022.12.02--15,single_execution,no_advisor,Mission 2,Process Scale,2.096874,0.000000e+00
5,2022.12.02--15,single_execution,no_advisor,Mission 2,Satisfaction,0.447890,4.099520e-215
37,2022.12.02--15,single_execution,human_advisor,Mission 1,Process Scale,0.024206,4.350153e-02
72,2022.12.02--15,single_execution,tomcat_advisor,Mission 1,Score,0.031814,1.224667e-02
74,2022.12.02--15,single_execution,tomcat_advisor,Mission 1,Satisfaction,1.304633,0.000000e+00
75,2022.12.02--15,single_execution,tomcat_advisor,Mission 2,Score,5.329812,0.000000e+00
76,2022.12.02--15,single_execution,tomcat_advisor,Mission 2,Process Scale,2.638571,0.000000e+00
77,2022.12.02--15,single_execution,tomcat_advisor,Mission 2,Satisfaction,2.626940,0.000000e+00
78,2022.12.02--15,single_execution,tomcat_advisor,All Missions,Score,3.580305,0.000000e+00


# Hypothesis III: Intervening on team communication predicts higher coordination and outcome measures


Compare mission by mission whether ToMCAT trials had higher coordination and outcome measures than others.

In [155]:
outcome_measures_labels = ["Score", "Process Scale", "Satisfaction"]

# Compare the second group against the first
groups = [
    ("no_advisor", "human_advisor"),
    ("no_advisor", "tomcat_advisor"),
    ("human_advisor", "tomcat_advisor"),
]

results_list = []
for model in models:
    for i, mission in enumerate(["Mission 1", "Mission 2", "All Missions"]):
        for condition1, condition2 in groups:
            dataset1 = datasets[condition1][i]
            inferences1 = load_model_inferences(condition1, model["ref_date"], model["training_type"])            
            coord1 = np.array([AGGR_FN(summary.coordination_mean) for summary in inferences1[i]])

            dataset2 = datasets[condition2][i]
            inferences2 = load_model_inferences(condition2, model["ref_date"], model["training_type"])            
            coord2 = np.array([AGGR_FN(summary.coordination_mean) for summary in inferences2[i]])
            
            # Comparing coordination. Outcome measures were compared in another notebook.
            results_entry = model.copy()
            results_entry.update({
                "condition1": condition1,
                "condition2": condition2,
                "mission": mission
            })
            results_entry.update(compare_conditions(coord2, coord1))
            results_list.append(results_entry)

df_h3 = pd.DataFrame.from_dict(results_list)
df_h3.head()

,ref_date,training_type,condition1,condition2,mission,cohens_d,p_val
0,2022.12.02--15,single_execution,no_advisor,human_advisor,Mission 1,0.307606,0.239131
1,2022.12.02--15,single_execution,no_advisor,tomcat_advisor,Mission 1,-0.549728,0.894645
2,2022.12.02--15,single_execution,human_advisor,tomcat_advisor,Mission 1,-0.826947,0.967306
3,2022.12.02--15,single_execution,no_advisor,human_advisor,Mission 2,0.248999,0.265545
4,2022.12.02--15,single_execution,no_advisor,tomcat_advisor,Mission 2,0.551135,0.090475


In [156]:
df_h3[(df_h3["p_val"] <= 0.05)]

,ref_date,training_type,condition1,condition2,mission,cohens_d,p_val
9,2022.12.04--12,single_execution_no_self_dep,no_advisor,human_advisor,Mission 1,3.750736,5.898395e-09
12,2022.12.04--12,single_execution_no_self_dep,no_advisor,human_advisor,Mission 2,2.368458,1.115005e-06
15,2022.12.04--12,single_execution_no_self_dep,no_advisor,human_advisor,All Missions,2.468181,6.392857e-12
31,2022.12.05--09,single_execution_pitch_only,no_advisor,tomcat_advisor,Mission 2,0.941234,1.348381e-02
32,2022.12.05--09,single_execution_pitch_only,human_advisor,tomcat_advisor,Mission 2,0.912112,1.574293e-02


## Hypothesis IV: Coordination in the second mission is higher than in the first mission

In [195]:
def get_paired_coordination(mission1_dataset: LatentVocalicsDataset, mission2_dataset: LatentVocalicsDataset, 
                            mission1_coordination: np.ndarray, mission2_coordination: np.ndarray):
    """
    Gets coordination for the same teams in mission 1 and 2. Ignores data from
    trials with only one mission.
    """
    
    mission1 = {}
    mission2 = {}
    mission1_trial_numbers = []
    
    for i in range(mission1_dataset.num_trials):
        mission_trial_number = int(mission1_dataset.series[i].uuid[1:])
        mission1[mission_trial_number] = mission1_coordination[i]
        
    for i in range(mission2_dataset.num_trials):
        mission_trial_number = int(mission2_dataset.series[i].uuid[1:])
        if mission_trial_number - 1 in mission1:
            # Only add if exists an entry for mission 1
            mission1_trial_numbers.append(mission_trial_number - 1)
        
            mission2[mission_trial_number] = mission2_coordination[i]
    
    coord1 = []
    coord2 = []
    for mission1_trial_number in mission1_trial_numbers:
        coord1.append(mission1[mission1_trial_number])
        coord2.append(mission2[mission1_trial_number + 1])
    
    return np.array(coord1), np.array(coord2)

In [200]:
results_list = []
for model in models:    
    for condition in ["no_advisor", "human_advisor", "tomcat_advisor", "all_advisors"]:
        inferences = load_model_inferences(condition, model["ref_date"], model["training_type"])            
        
        dataset1 = datasets[condition][0]
        dataset2 = datasets[condition][1]
        coord1 = np.array([AGGR_FN(summary.coordination_mean) for summary in inferences[0]])          
        coord2 = np.array([AGGR_FN(summary.coordination_mean) for summary in inferences[1]])        
        
        coord1, coord2 = get_paired_coordination(dataset1, dataset2, coord1, coord2)
        
        # Comparing coordination. Outcome measures were compared in another notebook.
        results_entry = model.copy()
        results_entry.update({
            "condition": condition
        })
        
        results_entry.update(compare_conditions(coord2, coord1, paired=True))
        results_list.append(results_entry)

df_h4 = pd.DataFrame.from_dict(results_list)
df_h4.head()

,ref_date,training_type,condition,cohens_d,p_val
0,2022.12.02--15,single_execution,no_advisor,-0.690918,0.967720
1,2022.12.02--15,single_execution,human_advisor,-0.760075,0.981983
2,2022.12.02--15,single_execution,tomcat_advisor,0.633299,0.081234
3,2022.12.02--15,single_execution,all_advisors,-0.005444,0.509818
4,2022.12.04--12,single_execution_no_self_dep,no_advisor,-0.856219,0.987129


In [201]:
df_h4[(df_h4["p_val"] <= 0.05)]

,ref_date,training_type,condition,cohens_d,p_val
10,2022.12.04--22,single_execution_intensity_only,tomcat_advisor,0.738191,0.028151
14,2022.12.05--09,single_execution_pitch_only,tomcat_advisor,0.916795,0.039697


In [204]:
with open(f"/Users/paulosoares/data/study-3_2022/datasets/4_features/no_advisor/all_missions_dataset.pkl", "rb") as f:
    test = pickle.load(f)

In [211]:
test.observed_vocalics[0, :, 205:210]

array([[ 0.        , -1.67547018,  0.        ,  0.        ,  0.        ],
       [ 0.        , -0.70372479,  0.        ,  0.        ,  0.        ],
       [ 0.        , -1.46554556,  0.        ,  0.        ,  0.        ],
       [ 0.        , -1.74234776,  0.        ,  0.        ,  0.        ]])